<hr style="border: 0.5px dashed #000;">

## Create CSV Output

### 1) Setup wallet and test funds<br>
**1.1) Generate a new private key**<br>
`bx seed ` creates 192 bytes of entropy. <br>
`bx ec-new ` derives 256bit private key from entropy.



In [96]:
%%bash

bx seed | bx ec-new

9273838e8af770d5a7ec54585e9846a03ecc9a3d11bda75c18fede3e0a6dacdc


**1.2 Save your new private key here:**<br>
Will be used later for signing.<br>
`e60c1ee5fb938aa041c15594b0df6ffbedbfc33e4aaf231b7eb4dbf18c464d8e`

**1.3 Derive public key**<br>
`bx ec-to-public [32-byte private key]`<br>

In [97]:
%%bash

bx ec-to-public e60c1ee5fb938aa041c15594b0df6ffbedbfc33e4aaf231b7eb4dbf18c464d8e

02e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31


**1.4 Derive public key hash**<br>
`bx sha256 [33-byte public key] | bx ripemd160`

In [98]:
%%bash

bx sha256 02e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31 | bx ripemd160

42a8bcd5f61fa17adbe800de6382f24bd1db0d7f


**1.5 Derive testnet address**<br>
Add version prefix for mainnet/testnet<br>
* `6f` in hex
* `111` in decimal
* `[6f][32-byte public key hash]`

Wrap with version prefix & checksum: <br>
* `[6f][32-byte public key hash][4-byte checksum]`
* bx command: `bx wrap-encode -v [version(decimal)] [public key hash]`

In [1]:
%%bash

bx wrap-encode -v 111 42a8bcd5f61fa17adbe800de6382f24bd1db0d7f

6f42a8bcd5f61fa17adbe800de6382f24bd1db0d7fe486d55f


Encode in base58:<br>
* `bx base58-code [address in base16]`
* For testnet address, begins with `m` or `n` 

In [2]:
%%bash

bx base58-encode 6f42a8bcd5f61fa17adbe800de6382f24bd1db0d7fe486d55f

mmbR2p2QQULJBKBHoEFYDByjdSV9WhKJ3Q


**1.5 Send test coins funds to your testnet address**<br>
Note your spendable UXTO's below...<br>
* UXTO 0: 
    * TXID: `423f3180d4a0f95c925553beef5e2fec7a4f19f9632c8294575ac450ff5b6b89`<br>
    * Index: `0`
    * Amount: `130000000`
* UXTO 1: 
    * TXID: `f91f383904ec4a6fd4902ebbb3b8f2d049c2ed24aa441d70bd3dbf2a077dcbe6`<br>
    * Index: `1`
    * Amount: `65000000`
* UXTO ...  
    * TXID: `...`<br>
    * Index: `...`

<hr style="border: 0.5px dashed #000;">

### 2) Encode transaction with CSV encumbered output<br>

**2.1) Encode relative locktime for output script**<br>
<br>Example:
* Decimal: `1384968`
* 4-Byte Hex (Big Endian): `00152208`
* 4-Byte Hex (Little Endian): `08221500`

In [120]:
%%bash
printf '%x\n' 1384968

152208


**Note:** Don't forget to flip byte-order (4-Bytes) for output script.

**2.3) Encode CLTV-encumbered P2PKH output script**
* **`[locktime]`**
* **`CHECKLOCKTIMEVERIFY`**
* `DROP`
* `DUP`
* `HASH160`
* `[public key hash]`
* `EQUALVERIFY`
* `CHECKSIG`

BX script-serialisation command:
`bx script-encode [bitcoin script]`

In [121]:
%%bash
bx script-encode \
"[08221500] CHECKLOCKTIMEVERIFY DROP DUP HASH160 [42a8bcd5f61fa17adbe800de6382f24bd1db0d7f] EQUALVERIFY CHECKSIG"

0408221500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac


**2.4) Determine output satoshi amount**<br>
* Assume `700 satoshi` transaction fee
* Output amount = `[UXTO spendable amount]`-`700`

In [125]:
%%bash
expr 130000000 - 700

129999300


**2.5) Encode transaction without output script**<br>
* `bx tx-encode -i [prevout txid:index:sequence] -o [output script:amount]`
* Input sequence defaults to `ffffffff`/`4294967295`

In [127]:
%%bash

bx tx-encode \
-i d6c4f5ea730a5e35ef126510a5542b01b7013d7601a5d741301348ef3cfae870:1:4294967295 \
-o 04fa211500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac:129999300

010000000170e8fa3cef48133041d7a501763d01b7012b54a5106512ef355e0a73eaf5c4d60100000000ffffffff01c4a1bf07000000002004fa211500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac00000000


Note: The input script field is initialised with `[00]`

Create Endorsement:
bx input-sign [privKey][prevOutScript][tx]

### 3) Endorsing the transaction

**3.1) Signing the transaction with the private key**
* Transaction is signed with `previous output script` set in the `input script field`
* `bx input-sign` `[private key]` `["previous output script"]` `[encoded transaction]`
    * Outputs DER encoded signature

In [180]:
%%bash
# bx input-sign \
# [private key] \
# "previous output script" \
# encoded transaction (empty input script)

bx input-sign \
e60c1ee5fb938aa041c15594b0df6ffbedbfc33e4aaf231b7eb4dbf18c464d8e \
"dup hash160 [42a8bcd5f61fa17adbe800de6382f24bd1db0d7f] equalverify checksig" \
010000000170e8fa3cef48133041d7a501763d01b7012b54a5106512ef355e0a73eaf5c4d60100000000ffffffff01c4a1bf07000000002004fa211500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac00000000

3045022100987522f31b627a5af943bba86be002fa980841a2b4796964c50fa6009cc44bd502202a0ccf99a93d33055dff5586898b3ca1733889f1ee322ec5f01363cb796491c501


**3.2) Set the input script into the finalised transaction**
* The input script for the previous p2pkh output script is `[endorsement]` `[public key]`
* `bx input-set` `"input script"` `[encoded transaction]`

In [140]:
%%bash
# bx input-set \
# "input script" \
# [previous output script] \

bx input-set \
"[3045022100987522f31b627a5af943bba86be002fa980841a2b4796964c50fa6009cc44bd502202a0ccf99a93d33055dff5586898b3ca1733889f1ee322ec5f01363cb796491c501] [02e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31]" 
010000000170e8fa3cef48133041d7a501763d01b7012b54a5106512ef355e0a73eaf5c4d60100000000ffffffff01c4a1bf07000000002004fa211500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac00000000

010000000170e8fa3cef48133041d7a501763d01b7012b54a5106512ef355e0a73eaf5c4d6010000006b483045022100987522f31b627a5af943bba86be002fa980841a2b4796964c50fa6009cc44bd502202a0ccf99a93d33055dff5586898b3ca1733889f1ee322ec5f01363cb796491c5012102e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31ffffffff01c4a1bf07000000002004fa211500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac00000000


**3.3) Validate the finalised transaction**
* `bx validate-tx` `[encoded transaction]`

In [139]:
%%bash

bx validate-tx \
010000000170e8fa3cef48133041d7a501763d01b7012b54a5106512ef355e0a73eaf5c4d6010000006b483045022100987522f31b627a5af943bba86be002fa980841a2b4796964c50fa6009cc44bd502202a0ccf99a93d33055dff5586898b3ca1733889f1ee322ec5f01363cb796491c5012102e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31ffffffff01c4a1bf07000000002004fa211500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac00000000

matching transaction with unspent outputs


In [1]:
%%bash
bx tx-decode -f json \
010000000170e8fa3cef48133041d7a501763d01b7012b54a5106512ef355e0a73eaf5c4d6010000006b483045022100987522f31b627a5af943bba86be002fa980841a2b4796964c50fa6009cc44bd502202a0ccf99a93d33055dff5586898b3ca1733889f1ee322ec5f01363cb796491c5012102e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31ffffffff01c4a1bf07000000002004fa211500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac00000000 \
|jq ".transaction.hash"

"918c9e1acf52698ea328df65c134ba1c5fdd119aea476ca6b934bbd03a6e3596"


**3.4) Broadcast transaction with CLTV encumbered output**
* `bx send-tx` `[encoded transaction]`

In [2]:
%%bash

bx send-tx \
010000000170e8fa3cef48133041d7a501763d01b7012b54a5106512ef355e0a73eaf5c4d6010000006b483045022100987522f31b627a5af943bba86be002fa980841a2b4796964c50fa6009cc44bd502202a0ccf99a93d33055dff5586898b3ca1733889f1ee322ec5f01363cb796491c5012102e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31ffffffff01c4a1bf07000000002004fa211500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac00000000

matching transaction with unspent outputs


<hr style="border: 0.5px dashed #000;">

## Spend CLTV encumbered UXTO

### 4) Encode CLTV spending transaction
**4.1) Encode p2pkh output script**

In [3]:
!bx script-encode "DROP DUP HASH160 [42a8bcd5f61fa17adbe800de6382f24bd1db0d7f] EQUALVERIFY CHECKSIG"

7576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac


**4.2) Encode transaction with locktime & set input sequence**<br>
* Reference Previous Outpoint<br>
* Transaction locktime field must be `>=` locktime in CLTV output being spent.
* Input sequence msut be less than maximum, so `fffffffe`/`4294967294`

In [4]:
%%bash
bx tx-encode \
-l 1384968 \
-i 918c9e1acf52698ea328df65c134ba1c5fdd119aea476ca6b934bbd03a6e3596:0:4294967294 \
-o 7576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac:129998600


010000000196356e3ad0bb34b9a66c47ea9a11dd5f1cba34c165df28a38e6952cf1a9e8c910000000000feffffff01089fbf07000000001a7576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac08221500


### 5) Endorse CLTV spending transaction
**5.1) Sign transaction (empty input script)**

In [5]:
%%bash 
bx input-sign e60c1ee5fb938aa041c15594b0df6ffbedbfc33e4aaf231b7eb4dbf18c464d8e \
"[fa211500] CHECKLOCKTIMEVERIFY DROP DUP HASH160 [42a8bcd5f61fa17adbe800de6382f24bd1db0d7f] EQUALVERIFY CHECKSIG" \
010000000196356e3ad0bb34b9a66c47ea9a11dd5f1cba34c165df28a38e6952cf1a9e8c910000000000feffffff01089fbf07000000001a7576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac08221500

3045022100c7a966f9eba206a6a59220defb4dc1801171bf5be00dff9fa5f5244e7678ed900220055a61e6e9b100eb57238fb6e7a214dbb53a1e459617c6ba48c71daacd6b0cc101


**5.2) Set endorsement into final transaction**

In [6]:
%%bash
bx input-set "[3045022100c7a966f9eba206a6a59220defb4dc1801171bf5be00dff9fa5f5244e7678ed900220055a61e6e9b100eb57238fb6e7a214dbb53a1e459617c6ba48c71daacd6b0cc101] [02e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31]" \
010000000196356e3ad0bb34b9a66c47ea9a11dd5f1cba34c165df28a38e6952cf1a9e8c910000000000feffffff01089fbf07000000001a7576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac08221500


010000000196356e3ad0bb34b9a66c47ea9a11dd5f1cba34c165df28a38e6952cf1a9e8c91000000006b483045022100c7a966f9eba206a6a59220defb4dc1801171bf5be00dff9fa5f5244e7678ed900220055a61e6e9b100eb57238fb6e7a214dbb53a1e459617c6ba48c71daacd6b0cc1012102e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31feffffff01089fbf07000000001a7576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac08221500


**5.3) Validate the finalised transaction**

In [7]:
%%bash
bx validate-tx \
010000000196356e3ad0bb34b9a66c47ea9a11dd5f1cba34c165df28a38e6952cf1a9e8c91000000006b483045022100c7a966f9eba206a6a59220defb4dc1801171bf5be00dff9fa5f5244e7678ed900220055a61e6e9b100eb57238fb6e7a214dbb53a1e459617c6ba48c71daacd6b0cc1012102e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31feffffff01089fbf07000000001a7576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac08221500

The transaction is valid.
